In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format='retina'

In [ ]:
import os
import json

from tqdm import tqdm
import numpy as np
import scipy.spatial
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import anndata
import scanpy as sc

In [ ]:
adata_bulk = sc.read_h5ad('/GPUData_xingjie/SCMG/hESC_perturb_seq/pseudo_bulk.h5ad')
adata_bulk

In [ ]:
# Mask out the direct target genes
for i in range(adata_bulk.shape[0]):
    pg = adata_bulk.obs['perturbed_gene'].iloc[i]
    
    if pg in adata_bulk.var_names:
        adata_bulk.X[i, adata_bulk.var_names.get_loc(pg)] = 0

hv_genes = list(adata_bulk.var[(np.abs(adata_bulk.X) > 0.2).sum(axis=0) > 1]['gene_name'])

print(len(hv_genes))

In [ ]:
adata = sc.read_h5ad('/GPUData_xingjie/SCMG/hESC_perturb_seq/adata_single_gene_pert.h5ad')
adata

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [ ]:
adata = adata[:, hv_genes].copy()
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver='arpack')
adata

In [ ]:
sc.pp.neighbors(adata, n_neighbors=20)

In [ ]:
sc.tl.leiden(adata)

In [ ]:
sc.tl.paga(adata)
sc.pl.paga(adata, plot=True)

In [ ]:
sc.tl.umap(adata, init_pos='paga')
#sc.tl.umap(adata)

In [ ]:
adata.obs['umap_x'] = adata.obsm['X_umap'][:, 0]
adata.obs['umap_y'] = adata.obsm['X_umap'][:, 1]
adata.obs.to_csv('adata_obs_l1.csv')

In [ ]:
sc.pl.umap(adata, color='leiden', legend_loc='on data')